In [85]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch_geometric.data import Data,Dataset

total_parms_new = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params_new.csv',index_col=0)
total_parms = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params.csv',index_col=0)

from typing import List, Literal, Sequence, Union


dataset_list = ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']
def get_datasetwise_df(df:pd.DataFrame,name=None):
    if not name:
        dflist = []
        for i in ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']:
            dflist.append(df[df['Dataset']==i])
        return dflist
    else:
        return df[df['Dataset']==name]

def get_f1accs(df: Union[pd.DataFrame, List[pd.DataFrame]]):
    metrics = ['Vanilla test acc','Vanilla test f1','Shadow test acc','Shadow test f1','Norm bound','Noise scale','MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse','Vanilla runtime per']
    criticals = ['Norm bound','Noise scale']
    if isinstance(df,list):
        res = []
        for i in df:
            res.append(i[metrics].max())
        return res
    else:
        return df[metrics].max()
    

total_parms_new.columns

Index(['Dataset', 'Vanilla train acc', 'Vanilla train loss',
       'Vanilla train f1', 'Vanilla test acc', 'Vanilla test loss',
       'Vanilla test f1', 'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate', 'MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse',
       'MIA confidence thr', 'MIA seed', 'Vanilla runtime per',
       'Shadow runtime per', 'Epsilon', 'Delta', 'Dp', 'Rdp', 'Ldp',
       'Norm bound', 'Noise scale', 'Sampler', 'Sampler batchsize',
       'Occurance k', 'Cluster numparts', 'Saint rootnodes',
       'Saint samplecoverage', 'Saint walklenth', 'Epochs', 'Shadow epochs',
       'Num val', 'Num test', 'Layers', 'Hidden dims', 'Learning rate',
       'Shadow learning rate', 'Dropout', 'Activation', 'Early stopping',
       'Patience', 'Optim type'],
      dtype='object')

In [86]:
def baseline(df:pd.DataFrame):
    clean_adam = df[(df['Dp']==True)&(df['Rdp']==False)&(df['Ldp']==False)&(df['Optim type']=='adam')]
    clean_sgd = df[(df['Dp']==True)&(df['Rdp']==False)&(df['Ldp']==False)&(df['Optim type']=='sgd')]
    return clean_adam,clean_sgd

In [87]:
dp_adam,dp_sgd = baseline(total_parms_new)

In [88]:
dp_adam

,Dataset,Vanilla train acc,Vanilla train loss,Vanilla train f1,Vanilla test acc,Vanilla test loss,Vanilla test f1,Shadow train acc,Shadow train loss,Shadow train f1,...,Num test,Layers,Hidden dims,Learning rate,Shadow learning rate,Dropout,Activation,Early stopping,Patience,Optim type
10,cora,0.1511,1.9439,0.1511,0.1461,1.9431,0.1461,0.9984,0.0296,0.9984,...,0.45,2,256,0.001,0.001,0.0,relu,True,100,adam
11,cora,0.1396,1.9461,0.1396,0.1166,1.9576,0.1166,0.9951,0.0263,0.9951,...,0.45,2,256,0.001,0.001,0.0,relu,True,100,adam
12,cora,0.1314,1.9493,0.1314,0.1002,1.9502,0.1002,0.9885,0.0441,0.9885,...,0.45,2,256,0.001,0.001,0.0,relu,True,100,adam
13,cora,0.1199,1.9492,0.1199,0.1248,1.9524,0.1248,0.9967,0.0196,0.9967,...,0.45,2,256,0.001,0.001,0.0,relu,True,100,adam
14,cora,0.1806,1.9399,0.1806,0.1724,1.9391,0.1724,0.9967,0.0345,0.9967,...,0.45,2,256,0.001,0.001,0.0,relu,True,100,adam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,cora,0.1166,1.9467,0.1166,0.1100,1.9576,0.1100,0.9984,0.0133,0.9984,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
756,cora,0.1314,1.9561,0.1314,0.1297,1.9496,0.1297,0.9984,0.0144,0.9984,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
757,cora,0.1379,1.9537,0.1379,0.1478,1.9537,0.1478,0.9967,0.0127,0.9967,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
758,cora,0.1494,1.9316,0.1494,0.1264,1.9281,0.1264,0.9967,0.0166,0.9967,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam


In [89]:
clean_list = get_datasetwise_df(dp_adam)

clean_res_dfs = get_f1accs(clean_list)
clean_res = {}
for i in range(len(dataset_list)):
    clean_res[dataset_list[i]] = clean_res_dfs[i].to_dict()
pd.DataFrame(clean_res).T

,Vanilla test acc,Vanilla test f1,Shadow test acc,Shadow test f1,Norm bound,Noise scale,MIA mlp,MIA svm,MIA ranfor,MIA logi,MIA ada,MIA confidence mse,Vanilla runtime per
cora,0.5928,0.5928,0.8243,0.8243,5.0,4.0,0.5033,0.5082,0.5238,0.5386,0.5213,0.6026,0.6904
citeseer,0.3128,0.3128,0.6992,0.6992,2.0,3.0,0.5000,0.5013,0.5000,0.5142,0.5000,0.5349,0.2614
pubmed,0.4833,0.4833,0.8530,0.8530,2.0,3.0,0.5001,0.5014,0.5036,0.5013,0.5004,0.5107,0.2691
computers,0.6332,0.6332,0.9053,0.9053,2.0,3.0,0.5110,0.5086,0.5047,0.5153,0.5062,0.5156,0.2570
photo,0.7337,0.7337,0.9395,0.9395,2.0,3.0,0.5067,0.5177,0.5142,0.5189,0.5096,0.5299,0.2585
cs,0.5124,0.5124,0.9069,0.9069,2.0,3.0,0.5004,0.5012,0.5002,0.5113,0.5004,0.5113,0.2745
physics,0.6932,0.6932,0.9521,0.9521,2.0,3.0,0.5001,0.5031,0.5001,0.5091,0.5005,0.5082,0.2738
reddit,0.4095,0.4095,0.9188,0.9188,2.0,1.0,0.5032,0.5045,0.5033,0.5084,0.5040,0.5093,0.2375
github,0.8054,0.8054,0.8628,0.8628,2.0,1.0,0.5041,0.5044,0.5054,0.5024,0.5041,0.5110,0.2207
flickr,0.3703,0.3703,0.4352,0.4352,2.0,1.0,0.5025,0.5047,0.5043,0.5086,0.5061,0.5084,0.2314


In [90]:
cora = clean_list[0]


In [91]:
def dp_drop(df:pd.DataFrame):
    labels = ['Vanilla train acc','Vanilla train loss','Vanilla train f1',
          'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate','MIA seed','Shadow runtime per', 'Num val', 'Num test', 'Layers', 'Hidden dims','Activation','Shadow learning rate','Dropout','Sampler', 'Sampler batchsize',
       'Occurance k', 'Cluster numparts', 'Saint rootnodes',
       'Saint samplecoverage', 'Saint walklenth','Rdp','Ldp']
    return df.drop(columns=labels)



In [92]:
def get_res_dp(cora:pd.DataFrame):
    maxx = ['Vanilla test acc','Vanilla test f1']
    meann = ['Vanilla runtime per','MIA mlp', 'MIA svm','MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse','Epsilon','Delta']
    minn = ['Epsilon','Delta']
    res = {}
   
    for k in maxx:
        res[k] = cora.max()[k]
    for k in meann:
        res[k] = cora.mean(numeric_only=True)[k]
    for k in minn:
        res[k] = cora.min()[k]
    return res

In [93]:
cora = dp_drop(clean_list[1])
cora_ns1 = cora[cora['Noise scale']==1]
get_res_dp(cora_ns1)

{'Vanilla test acc': 0.3128,
 'Vanilla test f1': 0.3128,
 'Vanilla runtime per': 0.18936000000000003,
 'MIA mlp': 0.5,
 'MIA svm': 0.5000666666666667,
 'MIA ranfor': 0.5,
 'MIA logi': 0.49779666666666667,
 'MIA ada': 0.5,
 'MIA confidence mse': 0.51288,
 'Epsilon': 103.3026,
 'Delta': 0.1}

In [94]:
# cora = dp_drop(clean_list[2])
cora_ns1 = cora[cora['Noise scale']==2]
get_res_dp(cora_ns1)

{'Vanilla test acc': nan,
 'Vanilla test f1': nan,
 'Vanilla runtime per': nan,
 'MIA mlp': nan,
 'MIA svm': nan,
 'MIA ranfor': nan,
 'MIA logi': nan,
 'MIA ada': nan,
 'MIA confidence mse': nan,
 'Epsilon': nan,
 'Delta': nan}

In [95]:
# cora = dp_drop(clean_list[2])
cora_ns1 = cora[cora['Noise scale']==3]
get_res_dp(cora_ns1)

{'Vanilla test acc': 0.2326,
 'Vanilla test f1': 0.2326,
 'Vanilla runtime per': 0.25606,
 'MIA mlp': 0.5,
 'MIA svm': 0.5,
 'MIA ranfor': 0.5,
 'MIA logi': 0.5004,
 'MIA ada': 0.5,
 'MIA confidence mse': 0.5050799999999999,
 'Epsilon': 24.6359,
 'Delta': 0.1}

In [96]:
# cora = dp_drop(clean_list[0])
cora_ns1 = cora[cora['Noise scale']==4]
get_res_dp(cora_ns1)

{'Vanilla test acc': nan,
 'Vanilla test f1': nan,
 'Vanilla runtime per': nan,
 'MIA mlp': nan,
 'MIA svm': nan,
 'MIA ranfor': nan,
 'MIA logi': nan,
 'MIA ada': nan,
 'MIA confidence mse': nan,
 'Epsilon': nan,
 'Delta': nan}